In [1]:
from dotenv import load_dotenv
import chromadb
import os

load_dotenv('../.env.local')
storage_path = '../'+os.getenv('STORAGE_PATH')
print(storage_path)

../chromadb


In [2]:
from IPython.display import display, Markdown
def view_text_in_markdown(page_content):
    display(Markdown(page_content))

## Vector Embeddings

In [3]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma

In [4]:
embeddings = OllamaEmbeddings(model="nomic-embed-text", show_progress=True)

def get_vector_store(collection_name):
  persistent_client = chromadb.PersistentClient(path='../chromadb')
  return Chroma(client=persistent_client,
                embedding_function=embeddings,
                collection_name=collection_name)

In [5]:
# Load vector database
vector_db = get_vector_store('owner_manual')

/tmp/ipykernel_3885/3805566098.py:5: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  return Chroma(client=persistent_client,


In [6]:
retriever = vector_db.as_retriever()

In [7]:
retriever.invoke('What is this document about?')

OllamaEmbeddings:   0%|          | 0/1 [00:00<?, ?it/s]

OllamaEmbeddings: 100%|██████████| 1/1 [00:07<00:00,  7.35s/it]


[Document(metadata={'source': 'pdf_files/Owners_Manual-Ram_1500_25_Crew_Cab.pdf'}, page_content="Diagnostic Procedure Manuals\n\nDiagnostic Procedure Manuals are filled with diagrams, charts and detailed illustrations. These manuals\n\nmake it easy to find and fix problems on computer- controlled vehicle systems and features. They show\n\nexactly how to find and correct problems, using step-by- step troubleshooting and drivability procedures, proven diagnostic tests and a complete list of all tools and equipment.\n\nTo order a digital copy of your Service or Diagnostic Procedure manuals, visit:\n\nwww.techauthority.com (US and Canada). Owner's Manuals\n\nThese Owner's Manuals have been prepared with the assistance of service and engineering specialists to acquaint you with specific FCA vehicles."),
 Document(metadata={'source': 'pdf_files/Owners_Manual-Ram_1500_25_Crew_Cab.pdf'}, page_content='The pretensioners are triggered by the Occupant Restraint Controller (ORC). Like the air bags

## Retrieval

In [8]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

In [9]:
# LLM from Ollama
local_model = "mistral"
llm = ChatOllama(model=local_model)

In [10]:
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You will answer questions about information that can be found in the owner's manual of the RAM 1500 vehicle, model year 2025, Crew Cab version.
    Your task is to generate five different versions of the given user question to retrieve relevant documents
    from a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

In [11]:
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)

# RAG prompt
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [12]:
retrieve_docs = retriever.invoke('What is the vehicle line') # get relevant documents

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  7.50it/s]


In [14]:
# big doc
retrieve_docs

[Document(metadata={'source': 'pdf_files/Owners_Manual-Ram_1500_25_Crew_Cab.pdf'}, page_content='rear Gross Axle Weight Rating (GAWR), and Vehicle Identification Number (VIN). A Month-Day-Hour (MDH) number is included on this label and indicates the Month, Day and Hour of manufacture. The bar code that appears on the bottom of the label is your VIN.\n\nGross Vehicle Weight Rating (GVWR)\n\nThe GVWR is the total permissible weight of your vehicle including driver, passengers, vehicle, options and cargo. The label also specifies maximum capacities of front\n\nSTARTING AND OPERATING 185\n\nand rear GAWR. Total load must be limited so GVWR and front and rear GAWR are not exceeded.\n\nPayload\n\nThe payload of a vehicle is defined as the allowable load weight a truck can carry, including the weight of the driver, all passengers, options and cargo.\n\nGross Axle Weight Rating (GAWR)'),
 Document(metadata={'source': 'pdf_files/Owners_Manual-Ram_1500_25_Crew_Cab.pdf'}, page_content='IF YOU NEE

In [15]:
len(retrieve_docs)

12

In [16]:
view_text_in_markdown(retrieve_docs[1].page_content)

IF YOU NEED ASSISTANCE . Roadside Assistance... FCA US LLC Customer Assistance Center.. 355 FCA Canada Customer Care. Mexico.

Contents 7

355

Puerto Rico And US Virgin Islands ... Customer Assistance For The Hearing Or Speech Impaired (TDD/TTY). Service Contract . WARRANTY INFORMATION MOPAR® PARTS REPORTING SAFETY DEFECTS .... In The 50 United States And Washington, D.C. In Canada ORDERING AND ACCESSING ADDITIONAL OWNER’S INFORMATION ..... CHANGE OF OWNERSHIP OR ADDRESS. GENERAL INFORMATION.....

8 INTRODUCTION

INTRODUCTION

Dear Customer, Congratulations on the purchase of your new Ram vehicle. Be assured that it represents precision workmanship, distinctive styling, and high quality.

In [17]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [18]:
resposta = chain.invoke("Where is located the hazard flashers button?")
view_text_in_markdown(resposta)

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  7.94it/s]


 The provided texts do not mention the location of the hazard flasher button. However, it's usually found on the stalk control located near the driver's side door where the turn signals are also located. In some vehicles, it may be a separate button or combined with the turn signal lever. Always refer to your vehicle's specific owner's manual for accurate information.

In [19]:
resposta = chain.invoke("Please list all the support centers that assist button can connect")
view_text_in_markdown(resposta)

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  7.21it/s]


1. Brand Connect Customer Care (If available)
  2. Vehicle Customer Care
  3. Uconnect Customer Care

In [24]:
resposta = chain.invoke("Please list all the support centers that assist button can connect")
view_text_in_markdown(resposta)

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  5.57it/s]


1. Roadside Assistance
  2. Brand Connect Customer Care (If available)
  3. Vehicle Customer Care
  4. Uconnect Customer Care

In [20]:
resposta = chain.invoke("This is an owner manual of a vehicle. Can you specify which vehicle?")
view_text_in_markdown(resposta)

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  7.58it/s]


 Based on the provided context, the vehicle is a Ram 1500 Crew Cab.

In [21]:
resposta = chain.invoke("Can you specify which vehicle?")
view_text_in_markdown(resposta)

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  8.57it/s]


 The provided documents are all related to the Ram 1500 25 Crew Cab.